In [1]:
from itertools import permutations
import random
from copy import deepcopy
import numpy as np
from scipy.stats import norm
import math
import statistics

In [2]:
##############################################
### OFFLINE EVALUATION
##############################################

def compute_ERR(list):
    """
    Outputs the ERR measure of the rankinglist ([doc_id1, doc_id2, doc_id3]).
    """
    ERR = 0
    for r in range(len(list)):
        theta_r = compute_click_probability(get_relevance(list[r]))
        prod = 1
        for i in range(r):
            prod *= (1-compute_click_probability(get_relevance(list[i])))
        ERR += prod*theta_r*1/(r+1)
    return ERR

In [3]:
def compute_click_probability(rel):
    """
    Outputs the click-probability theta.
    """
    rel_max = 1
    return (2**rel - 1)/2**rel_max

In [4]:
def create_ranking_pairs():
    """
    TODO: maybe we want to exclude some pairs?
    Outputs a list of tuples with all possible ranking pairs from a pool of 12 document ids.
    """
    list_of_pairs = []
    perm = list(permutations(range(12),3))
    for i in perm:
        for j in perm:
            pair = (list(i),list(j))
            list_of_pairs.append(pair)
    return list_of_pairs

In [5]:
def get_relevance(doc):
    """
    Outputs the relevance of the document, given as integer.
    """
    if (doc<0 or doc>11):
        raise Exception('Document id should be from 0 to 12!')

    if doc<6:
        return 0
    else:
        return 1
    
def get_relevance_list(docs):
    return [get_relevance(doc) for doc in docs]

In [6]:
def divide_pairs_over_bins(list_pairs):
    """
    Divides a list of pairs over 10 bins according to the delta ERR.
    """
    dERRs = [[] for i in range(10)]
    for i in list_pairs:
        deltaERR = compute_ERR(i[0]) - compute_ERR(i[1])
        #Keep all the positive delta ERRs and put them in the respective bin
        if deltaERR > 0:
            if deltaERR < 0.1:
                dERRs[0].append(i)
            else:
                bin = str(deltaERR*10)
                dERRs[int(bin[0])].append(i)
    return dERRs

In [7]:
pairs = create_ranking_pairs() #list of all distinct ranking pairs

In [8]:
deltaERRs = divide_pairs_over_bins(pairs) #list of all pairs (tuples of two lists) divided over the 10 bins

In [9]:
def coin_to_ranker(cointoss):
    if (cointoss == 0):
        return "E"
    if (cointoss == 1):
        return "P"
    else:
        raise Exception('This number should be either 0 or 1!')

In [10]:
## For testing purposes:
#lst = [7, 8, 9]
#print(compute_ERR(lst))

#print(divide_pairs_over_bins([([3,5,7],[1,2,3]),([6,8,11],[3,2,0]),([3,5,8],[1,2,3])]))

##############################################
### ONLINE EVALUATION
##############################################

## Interleaving
def td_interleave(pair):
    # Writing the pair as a list makes it mutable, which makes the rest easier to code.
    # If we do not want the original pair to be changed, we need to do a deepcopy
    new_pair = deepcopy(list([pair[0], pair[1]]))
    result = []
    for i in range(3):
        cointoss = random.randint(0, 1)
        result += [(new_pair[cointoss][0], coin_to_ranker(cointoss))]
        if (new_pair[cointoss][0] in new_pair[1 - cointoss]):
            new_pair[1 - cointoss].remove(new_pair[cointoss][0])
        new_pair[cointoss] = new_pair[cointoss][1:]
    return result

In [11]:
# The softmax takes the rank of a document as well as the maximal rank and returns
# the probability of choosing that document
def softmax(r, max_r):
    tau = 3
    ranks = range(1,max_r+1)
    normalizer = sum([rank**(-tau) for rank in ranks])
    return r**(-tau)/normalizer

In [12]:
def pr_interleave(pair):
    # Writing the pair as a list makes it mutable, which makes the rest easier to code.
    # If we do not want the original pair to be changed, we need to do a deepcopy
    new_pair = deepcopy(list([pair[0], pair[1]]))
    result = []
    sm = [[],[]]
    sm[0] = [softmax(r, 3) for r in range(1, 4)]
    sm[1] = [softmax(r, 3) for r in range(1, 4)]
    population = [[0, 1, 2], [0, 1, 2]]
    for i in range(3):
        cointoss = random.randint(0, 1)
        index = random.choices(population[cointoss], sm[cointoss])[0]
        sm[cointoss][index] = 0
        result += [(new_pair[cointoss][index], coin_to_ranker(cointoss))]
        for j in range(3):
            if (new_pair[cointoss][index] == new_pair[1 - cointoss][j]):
                sm[1 - cointoss][j] = 0       
    return result

In [13]:
## Simulating user clicks
def yandex_log_parser():
    """
    This function parses the Yandex Click Log File and yields which ranks are clicked in a session.
    """
    sessionID = 0
    links = [-1 for i in range(10)] #dummy list
    clicks = [0 for i in range(10)]
    list_clicks = []
    with open("YandexRelPredChallenge.txt") as f:
        for line in f:
            words = line.split()
            sessionID_old = sessionID
            sessionID = int(words[0])
            if sessionID>sessionID_old:
                list_clicks.append(clicks)
                clicks = [0 for i in range(10)]
            recordType = words[2]
            if recordType=="Q":
                links = [int(l) for l in words[5:]]
            elif recordType=="C":
                link_clicked = int(words[3])
                if link_clicked in links:
                    rank_clicked = links.index(link_clicked)
                    clicks[rank_clicked] =1
    return(list_clicks)

In [14]:
def em():
    """
    TODO: check whether it is correct
    Expectation-maximization method for determining the parameters alpha and gamma, using training data.
    Using the update rules from: https://clickmodels.weebly.com/uploads/5/2/2/5/52257029/mc2015-clickmodels.pdf
    """
    # Use this: https://github.com/markovi/PyClick/blob/master/pyclick/utils/YandexRelPredChallengeParser.py to understand what is going on
    # https://www.kaggle.com/c/yandex-personalized-web-search-challenge#logs-format
    gamma = [0.5 for x in range(10)] #book
    alpha = 0.2 #book, initial probability clicked if not relevant
    tolerance = 0.01
    max_iter = 100
    click_log = yandex_log_parser()
    for i in range(max_iter):
        total = [1 for x in range(10)]
        for j in click_log:
            for rank in range(len(j)):
                gamma_value = gamma[rank]/total[rank]
                alpha_value = alpha/sum(total)
                if j[rank]==1:
                    gamma[rank] += 1
                    alpha += 1
                else:
                    alpha += (1-gamma_value)*alpha_value/(1-gamma_value*alpha_value)
                    gamma[rank] += (1-alpha_value)*gamma_value/(1-gamma_value*alpha_value)
                total[rank] += 1
        # New alpha and gamma
        alpha = alpha/sum(total)
        for x in range(len(gamma)):
            gamma[x] = gamma[x]/total[x]
    return (alpha,gamma)

In [15]:
(alpha,gamma) = em()

In [16]:
theta = 0.5

In [17]:
print(alpha, gamma)

0.5153349591804152 [1.0, 1.0, 0.7827273817329445, 0.6039110707434906, 0.4478730151477848, 0.3901581322301723, 0.34738501037066255, 0.3231173617327105, 0.30932417965860853, 0.31685576254851017]


In [18]:
# It takes a ranked list l of relevance scores, and parameters alpha and gamma
# and outputs the probabilities that different elements are clicked in the end
def click_probabilities(l, alpha, gamma):
    return [abs(alpha - (1 - l[i]))*gamma[i] for i in range(3)]

In [19]:
l = [0, 0, 0]
print(click_probabilities(l, alpha, gamma))

[0.48466504081958484, 0.48466504081958484, 0.37936059841820435]


In [20]:
# Takes a list of relevance scores and parameters, and returns for each position whether it is clicked or not
# 1 means a click.
def produce_clicks(list, alpha, gamma):
    probabilities = click_probabilities(list, alpha, gamma)    
    return [np.random.binomial(1, probabilities[i]) for i in range(3)]

In [21]:
# Based on a click-probability theta, clicks are completely random here.
def produce_clicks_random(list, theta):
    return [np.random.binomial(1, theta) for i in range(3)]

In [42]:
# Takes an interleaved list and a click pattern and returns the winner
def decide_winner(interl, clicks):
    clicks_E = 0
    clicks_P = 0
    for i in range(3):
        if (clicks[i] == 1):
            if (interl[i][1] == "E"):
                clicks_E += 1
            elif (interl[i][1] == "P"):
                clicks_P += 1
    if (clicks_E > clicks_P):
        return "E"
    elif (clicks_E < clicks_P):
        return "P"
    else:
        return "NW"
    
# Returns winner of probabilistic interleaving according to the marginalised estimate
# Note that the probability of an assignment given an interleaved list is proportional
# to the probability of a list given an assignment, and thus only the latter is computed
def decide_winner_marg(interl, ranking_pair, clicks):
    prob_win = [0,0] # unnormalised 'probability' that system 0 or 1 wins
    
    # loop over assignments
    for assignment in [[a,b,c] 
                       for a in [0,1] 
                       for b in [0,1] 
                       for c in [0,1]]:
        prob_list = 1 # probability of interleaved list given assignment
        click_count = [0,0] # count how often each system's docs are clicked
        sm = [[softmax(r, 3) for r in range(1, 4)], 
              [softmax(r, 3) for r in range(1, 4)]]
        
        # loop over list positions
        for i in range(3):
            doc = interl[i][0]
            ass = assignment[i]
            cli = clicks[i]
            # count clicks
            click_count[ass] += cli
            # compute probability of list given assignment
            if doc not in ranking_pair[ass]:
                prob_list = 0
            else:
                index = ranking_pair[ass].index(doc)
                prob_list *= sm[ass][index]
                sm[ass][index] = 0
                sum_sm = sum(sm[ass])
                if sum_sm != 0:
                    sm[ass] = [s/sum_sm for s in sm[ass]]
            
        # assign win probability to appropriate system
        if click_count[0] != click_count[1]: # not a tie
            prob_win[click_count[1] > click_count[0]] += prob_list
        
#         print(assignment, prob_list)
#     print(prob_win)
    # return winner
    if (prob_win[1] > prob_win[0]):
        return "E"
    elif (prob_win[1] < prob_win[0]):
        return "P"
    else:
        return "NW"
       
# for testing
# il = [(1,"E"),(2,"P"),(3,"P")]
# rp = [[2,1,3],[1,2,3]]
# cl = [0,1,0]
# w=decide_winner_marg(il, rp, cl)
# print(w)

In [23]:
yandex_log = 0 # Variable containing the click log we use for determining the parameters alpha and gamma

In [122]:
## Simulation of Interleaving Experiment

# This determines the win_proportion of a single pair, while we do the interleaving and click-experiment
# 500 times. (500 can be changed later)
# The third argument specifies the function that determines the click_probabilities (position based or random)
def estimate_win_proportion(ranking_pair, interleave_function, click_function, alpha, gamma, theta):
    k = 500
    wins_E = 0
    wins_P = 0
    for i in range(k):
        interl = interleave_function(ranking_pair)
        relevance_list = get_relevance_list([elem[0] for elem in interl])
        if (click_function == produce_clicks):
            clicks = click_function(relevance_list, alpha, gamma)
        elif (click_function == produce_clicks_random):
            clicks = click_function(relevance_list, theta)
        winner = decide_winner(interl, clicks)
        if (winner == "E"):
            wins_E += 1
        elif (winner == "P"):
            wins_P += 1
            
    if (wins_E + wins_P > 0):
        return wins_E / (wins_E + wins_P)
    else:
        return 0.5
    
    
# Determines win proportion of experimental system based on the marginalised estimator of probabilistic interleaving
def estimate_win_proportion_marginalised(ranking_pair, click_function, alpha, gamma, theta):
    k = 500
    wins_E = 0
    wins_P = 0
    for i in range(k):
        # interleave 
        interl = td_interleave(ranking_pair)
        # simulate clicks
        relevance_list = get_relevance_list([elem[0] for elem in interl])
        if (click_function == produce_clicks):
            clicks = click_function(relevance_list, alpha, gamma)
        elif (click_function == produce_clicks_random):
            clicks = click_function(relevance_list, theta)
        # decide winning system
        winner = decide_winner_marg(interl, ranking_pair, clicks)
        if (winner == "E"):
            wins_E += 1
        elif (winner == "P"):
            wins_P += 1
            
    if (wins_E + wins_P > 0):
        return wins_E / (wins_E + wins_P)
    else:
        return 0.5

In [123]:
def compute_sample_size(p1):
    a = 0.05
    b = 0.1
    p0 = 0.5
    delta = abs(p1-p0)
    N_intermediate = ((norm.ppf(1 - a)*math.sqrt(p0*(1-p0)) + 
                      norm.ppf(1 - b)*math.sqrt(p1*(1 - p1)))
                      /delta)**2
    N = N_intermediate + 1/delta
    return math.ceil(N)

In [124]:
###############################################
###############################################
# Now we have everything we need and only need to do the simulation to be done.

In [138]:
def run_interleaving_experiment(deltaERRs, estimator, interleaver, click_function, alpha, gamma, theta):
    result = []
    counter = 0
    total = sum([len(d) for d in deltaERRs])
    for i in range(len(deltaERRs)):
        if (len(deltaERRs[i])> 0):
            Ns = []
            count_Ninf = 0 # counts how often N is infinite (p1<=0.5)
            for pair in deltaERRs[i]: 
                p1 = estimator(pair, interleaver, click_function, alpha, gamma, theta)
                counter += 1
                if counter % 10000 == 0:
                    print("{:.0f}%".format(counter/total*100))
                if (p1 > 0.5):
                    Ns += [compute_sample_size(p1)]
                else:
                    count_Ninf += 1
            if(len(Ns) > 0):
                minimum = min(Ns)
                maximum = max(Ns)
                median = statistics.median(Ns)
                result += [(minimum, median, maximum, count_Ninf/len(deltaERRs[i]))]
            else:
                result += [(None,None,None, 1)]
        else:
            result += [(None,None,None, None)]
    return result

In [ ]:
table = run_interleaving_experiment(deltaERRs, 
                                    estimate_win_proportion, 
                                    td_interleave, 
                                    produce_clicks, 
                                    alpha, gamma, theta)

1%
3%
4%
5%
7%
8%
9%
11%


In [ ]:
for el in table:
    print(el)

In [137]:
print("{:.0f}".format(11.1))

11
